In [2]:
import tweepy
import json
import math
from datetime import datetime, timedelta
from pprint import pprint
import csv
from tqdm import tqdm
import sys
import pandas as pd

api_details = {
"consumer_key" : "sxmfWBPf7D2RPj5xzF1sQEnVv",
"consumer_secret" : "ZaweN39f3Bc0BFa4vawZmDxkxG3zRwog5YdxLUpyFQWacPfRjy",
"access_token" : "1111593376804028416-Ghl8gXlGdQclqEqMwRRO3Kl8MNMq7a",
"access_token_secret" : "kU2N2hyzsWVXteD5wrVTLQaHnycRkQjXvMXGHSlviggZ4",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys = []
keys.append(api_details)

api_details = {
"consumer_key" : "DgenSt0OxyaCcMuuJyMCdkJGd",
"consumer_secret" : "o1lDfV4AG0tWufIHXGtjY6bNSuAdrSL9hJs4onZRDvEKNMS9t8",
"access_token" : "1111593376804028416-yhGKowC8Xpd8HNUDNeS7fjjKsZyreT",
"access_token_secret" : "bfJX8z1PrIcsui8W5CuUDBB48ii3pcA9TRKSMw4sadCdn",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

api_details = {
"consumer_key" : "xr4UfffLzPLCf1cEaqnApY5lQ",
"consumer_secret" : "DWjWu0VrD0uOuOmT0W4muH6CXai1pfHi6aqvGQHt2XOu1Yp9qw",
"access_token" : "1111593376804028416-zowDK4z6cLjWk5QOEUbFRxUzg1aYsJ",
"access_token_secret" : "m9c5cpiKu5WXHUhFLTTLFT7t9k7Nbryvt9qbCjfSSXka7",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

api_details = {
"consumer_key" : "9yJjSm8dTF3ZwCUg9jqKdryqF",
"consumer_secret" : "VCr4QQwuKpokF2DOOYRUb7jOQQrAubJrMzDJZrcKrRfqRVkiYp",
"access_token" : "1111593376804028416-Au6KVQjK8C10ufJ4l16ZUNrU6EMbEH",
"access_token_secret" : "UqrjjE5uyD0gMO1Pswsvy5oXwhe07hVhvLMJYVGQnfsZR",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

def doauth(index):
    auth = tweepy.OAuthHandler(keys[index]['consumer_key'], keys[index]['consumer_secret'])
    auth.set_access_token(keys[index]['access_token'], keys[index]['access_token_secret'])
    api = tweepy.API(auth)
    return api

idx = 0

try:
    api = doauth(idx)
except:
    idx += 1
    idx %= len(keys)
    api = doauth(idx)

In [2]:
df1 = pd.read_csv('./output/nuanced_hate.txt')
df2 = pd.read_csv('./dataset/Hate_Counter_Dataset.csv')

In [3]:
names=["Hate_ID" , "Counter_ID"]
df1.columns = names
df2.columns = names

In [4]:
df3 = pd.concat([df1, df2])

In [5]:
hate_tweets  = df3["Hate_ID"]
reply_tweets = df3["Counter_ID"]

In [6]:
orig_data = pd.read_csv("./dataset/NAACL_SRW_2016.csv")

In [7]:
orig_data.head()

,Tweet_ID,category
0,572342978255048705,racism
1,572341498827522049,racism
2,572340476503724032,racism
3,572334712804384768,racism
4,572332655397629952,racism


In [130]:
final_op = df3.join(orig_data.set_index('Tweet_ID'),on='Hate_ID')

In [131]:
final_op.to_csv("final_data_merged.csv",index=False)

In [2]:
df = pd.read_csv("final_data_merged.csv")

In [3]:
hate_id = df["Hate_ID"]
counter_id = df["Counter_ID"]

In [4]:
cnt = 0
n_keys = 3

hate_texts = []
counter_texts = []

hate_content = []
counter_content = []

for vals in hate_id:
    api = doauth(cnt%n_keys)
    cnt += 1
    print(cnt)
    try:
        mdata = api.get_status(vals,tweet_mode='extended')
        mvals = mdata._json
        hate_texts.append(mvals['full_text'])
        hate_content.append(mvals)
    except:
        print("FK")
        hate_texts.append("")
        hate_content.append("")
        
for vals in counter_id:
    api = doauth(cnt%n_keys)
    cnt += 1
    print(cnt)
    try:
        mdata = api.get_status(vals,tweet_mode='extended')
        mvals = mdata._json
        counter_texts.append(mvals['full_text'])
        counter_content.append(mvals)
    except:
        print("FK")
        counter_texts.append("")
        counter_content.append("")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


FK
1754
FK
1755
1756
1757
FK
1758
1759
FK
1760
FK
1761
FK
1762
1763
1764
1765
1766
1767
1768
1769
FK
1770
FK
1771
1772
1773
FK
1774
FK
1775
FK
1776
1777
FK
1778
FK
1779
1780
FK
1781
FK
1782
1783
1784
1785
1786
FK
1787
FK
1788
FK
1789
FK
1790
FK
1791
FK
1792
FK
1793
FK
1794
FK
1795
FK
1796
1797
1798
1799
1800
1801
1802
1803
1804
FK
1805
1806
1807
1808
1809
1810
1811
1812
1813
FK
1814
1815
FK
1816
FK
1817
1818
1819
FK
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
FK
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
FK
1845
FK
1846
FK
1847
FK
1848
FK
1849
1850
1851
1852
1853
1854
FK
1855
FK
1856
FK
1857
FK
1858
FK
1859
FK
1860
FK
1861
FK
1862
FK
1863
FK
1864
FK
1865
FK
1866
FK
1867
FK
1868
FK
1869
1870
FK
1871
FK
1872
1873
1874
1875
1876
FK
1877
FK
1878
FK
1879
FK
1880
FK
1881
FK
1882
1883
1884
1885
1886
1887
1888
1889
1890
FK
1891
1892
1893
1894
1895
1896
1897
1898
1899
FK
1900
FK
1901
FK
1902
1903
FK
1904
FK
1905
FK
1906
FK
1907
1908
1909
1910
1911
1912
1913

3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
FK
3179
3180
3181
3182
3183
3184
3185
FK
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
FK
3225
FK
3226
FK
3227
FK
3228
FK
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
FK
3244
3245
FK
3246
3247
3248
3249
3250
3251
325

4580
4581
FK
4582
4583
4584
FK
4585
4586
FK
4587
FK
4588
FK
4589
4590
4591
FK
4592
4593
4594
4595
4596
4597
4598
FK
4599
FK
4600
FK
4601
FK
4602
FK
4603
4604
4605
4606
FK
4607
4608
4609
4610
4611
4612
4613
FK
4614
4615
FK
4616
FK
4617
FK
4618
FK
4619
4620
4621
FK
4622
4623
4624
FK
4625
FK
4626
4627
4628
4629
4630
FK
4631
FK
4632
4633
FK
4634
FK
4635
FK
4636
4637
FK
4638
FK
4639
4640
FK
4641
4642
FK
4643
4644
FK
4645
FK
4646
FK
4647
4648
4649
FK
4650
4651
4652
FK
4653
4654
4655
4656
4657
FK
4658
4659
4660
4661
4662
FK
4663
4664
4665
4666
4667
FK
4668
FK
4669
4670
4671
4672
4673
4674
4675
4676
FK
4677
FK
4678
FK
4679
4680
4681
FK
4682
4683
FK
4684
FK
4685
4686
FK
4687
FK
4688
4689
FK
4690
4691
4692
4693
4694
FK
4695
FK
4696
4697
4698
FK
4699
4700
FK
4701
4702
4703
4704
4705
4706
FK
4707
4708
4709
4710
FK
4711
FK
4712
FK
4713
4714
FK
4715
FK
4716
FK
4717
FK
4718
FK
4719
FK
4720
4721
4722
4723
4724
4725
FK
4726
4727
4728
4729
FK
4730
4731
4732
4733
4734
4735
FK
4736
FK
4737
FK
4738
FK
4739

In [5]:
df["Hate_text"]  =  hate_texts
df["Counter_text"]  = counter_texts

In [6]:
df.to_csv("final_data_merged2.csv",index=False)

In [8]:
print((hate_content[0]))

{'created_at': 'Mon Mar 02 10:48:54 +0000 2015', 'id': 572347842456522752, 'id_str': '572347842456522752', 'full_text': 'Wish these blondes were in that How To Get Away With Murder show....#MKR', 'truncated': False, 'display_text_range': [0, 72], 'entities': {'hashtags': [{'text': 'MKR', 'indices': [68, 72]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2587278392, 'id_str': '2587278392', 'name': 'LilBeasy91', 'screen_name': 'lilbeastunleash', 'location': 'Gold Coast, Queensland', 'description': 'Psn - lilbeasty91 insta - lilbeasty91 Drunken gaming and Sassy reality tv tweeting is my bag baby.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 549, 'friends_count': 1878, 'listed_count

In [9]:
with open('./hate_content,json', 'w') as fout:
    json.dump(hate_content , fout)

In [10]:
with open('./counter_content.json', 'w') as fout:
    json.dump(counter_content , fout)

In [3]:
df1 = pd.read_csv("./dataset/abusive.csv")
df2 = pd.read_csv("./dataset/abusive_tweets_compile.csv")

In [5]:
df1.head()

,tweet_id,label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [10]:
df2.rename(columns={'Tweet_ID': 'tweet_id'}, inplace=True)
df2.head()

,Index,tweet_id,category,hate_users,creation_date
0,0,847503045961236482,+,@GummyBearDaddy,2017-03-30 17:37:26
1,1,848377709361287168,+,@Todd McGhee,2017-04-02 03:33:02
2,2,847600211224477696,+,@Mark Abadi,2017-03-31 00:03:32
3,3,849469625087799297,+,@brianna💋,2017-04-05 03:51:55
4,4,848295484217114624,+,@✨Whitney McGruder is editing her sequel✨,2017-04-01 22:06:18


In [12]:
final_op = df2.merge(df1)

In [16]:
final_op.label.value_counts()

normal     38027
abusive    11723
spam        8438
hateful     2450
Name: label, dtype: int64